# Generate introductions for statements

> This is just a quick test for generating background information for statements used in the upcoming Finnish local elections.

In [1]:
import json
import os
import sys
import re

# Get the absolute path to the parent directory (assumes this file is in 'condensation')
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

# autoreload modules
%load_ext autoreload
%autoreload 2

from typing import List
import pandas as pd
from chatbot_api.providers.openai import OpenAIProvider
from chatbot_api.base import Role, Message

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Generate

In [2]:
class StatementInfoGenerator:
    def __init__(self, llm_provider: OpenAIProvider, prompt_template: str):
        self.llm_provider = llm_provider
        self.PROMPT_TEMPLATE = prompt_template

    async def process_batch(self, statements: list[str]) -> list[tuple[str, str]]:
        '''
        Process a batch of statements and generate infos.
        '''
        results = []
        for statement in statements:
            print(f'Processing statement: {statement}...')
            prompt = self.PROMPT_TEMPLATE.format(statement=statement)
            info = await self.llm_provider.generate([Message(Role.USER, prompt)])
            results.append([statement, info])
        print('Done!')
        return results

In [ ]:
def build_prompt_template(data_path: str, n_examples: int) -> str:
    """
    Generate a prompt template for generating statement info using examples from the json file.
    NB. The examples currently deal with only task #2, the rest of the tasks are left for the LLM to handle by itself.
    """
    example_data = pd.read_json(data_path)
    
    prompt_template = """
# Ohjeet

1. Lue alla olevat esimerkit vaalikoneen väittämistä, joista käyttä voi olla joko samaa tai eri mieltä, sekä niihin liittyvät lyhyet taustatiedot.
2. Luo kohdassa 'Uusi väittämä' olevalle väittämälle samanlainen taustatietoteksti.
3. Luo väittämälle lisäksi lyhyt, muutaman lauseen teksti, joka kertoo, mikä väittämässä esitetyn asian tila on tällä hetkellä.
4. Jos väittämässä esiintyy termejä, jotka saattavat olla tavallisille ihmisille epäselviä, lisää yhden lauseen selitys jokaiselle termille.
5. Esitä nämä teksti alla olevan formaatin mukaisesti.

# Tulosformaatti

## [Väittämä]

### Taustatiedot

[Taustietoteksti]

### Nykytila

[Asian nykytila]

### Termit
- [Termi 1]: [Termin 1 selitys]
- [Termi 2]: [Termin 2 selitys]
- jne.

# Esimerkit taustietoteksteistä
"""
    for i in range(min(n_examples, len(example_data))):
      item = example_data.iloc[0]
      text = item['text']['fi']
      info = item['info']['fi']
      prompt_template += f"""

## Esimerkki {i+1}

- Väittämä: {text}

- Taustatiedot: {info}
"""

    prompt_template += """

# Uusi väittämä

Luo alla olevalle väittämälle ohjeissa määritetyt tekstit ja näytä ne tulosformaatin mukaisesti.

- Väittämä: {statement}
"""

    return prompt_template

In [ ]:
async def main():
    # Params
    n_examples = 20
    # The source data for examples of bg info
    examples_path = '../data/sources/opinionQuestions.json'
    # The statements we want to generate bg info for
    input_data = input_data = pd.read_csv('input/local-elections-2025-staments.tsv', sep='\t')

    # Read examples and build prompt template
    prompt_template = build_prompt_template(examples_path, n_examples)

    # Config
    api_key = os.getenv("OPENAI_API_KEY")
    model="gpt-4o-2024-11-20"
    openai_provider = OpenAIProvider(api_key, model)    
    processor = StatementInfoGenerator(openai_provider, prompt_template)

    # Setup paths
    output_path = os.path.join(parent_dir, 'statement_info', 'results', 'infos-v2.md')

    # Process statements
    results = await processor.process_batch(statements=input_data.statement.to_list()[:1])
    
    # Save to file'
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            for statement,response in results:
                f.write(f'{response.content}\n\n')
                f.write(f'(Total tokens: {response.usage.total_tokens})\n\n')
    except Exception as e:
        print(f'Error writing to file: {e}')

    return results

results = await main()

Processing statement: Äänestysikärajaa tulee laskea 16 ikävuoteen kunta- ja aluevaaleissa....
Done!


# Archived

## Read examples

In [2]:
example_data = pd.read_json('../data/sources/opinionQuestions.json')
example_data.head()

,_originalId,text,info,allowOpen,order,category,entityType,questionType,id,customData
0,QUESTION_pyiDjmLkPySDAi1Ucg6VeK,{'fi': 'Opintoihin kuuluvasta työharjoitteluis...,{'fi': '<p>Moniin opintoihin kuuluu työharjoit...,False,10000,[2],candidate,[1],1001,{'localized': {'fi': {'video': {'aspectRatio':...
1,QUESTION_bU9btBiuL4rKPqxqhM7AmK,{'fi': 'Nuorten mielenterveyskriisiin ratkaise...,{'fi': '<p>Terveyspolitiikka ja siihen kuuluva...,False,20000,[2],candidate,[1],1002,{'localized': {'fi': {'video': {'aspectRatio':...
2,QUESTION_dnRTPKjPufiZAyiHAu59HB,{'fi': 'Myös 16- ja 17-vuotiaiden pitää saada ...,{'fi': '<figure><img src='https://dfe4vsdvqdch...,False,30000,[2],candidate,[1],1003,{'localized': {'fi': {'video': {'aspectRatio':...
3,QUESTION_nhdw6WEgvor3V1VsLpi4mL,{'fi': 'Milloin EU:n pitää pyrkiä ilmastoneutr...,"{'fi': '<p>Ilmastoneutraalius tarkoittaa sitä,...",False,40000,[2],candidate,[9],1004,"{'vertical': True, 'localized': {'fi': {'video..."
4,QUESTION_6PrVdLYFepRjwtqU9gg4WL,"{'fi': 'EU:n ei tule ottaa yhteistä velkaa.', ...",{'fi': '<p>EU otti yhteistä velkaa vuonna 2020...,False,50000,[2],candidate,[1],1005,{'localized': {'fi': {'video': {'aspectRatio':...


In [3]:
item = example_data.iloc[0]
item['text']['fi'], item['info']['fi'], 

('Opintoihin kuuluvasta työharjoitteluista pitää aina maksaa palkkaa EU:ssa.',
 '<p>Moniin opintoihin kuuluu työharjoittelua. Se voi Suomessa olla tällä hetkellä palkallista tai palkatonta. Tähän vaikuttavat muun muassa opiskeltava ala ja sen työllisyystilanne. Jos harjoittelu on palkatonta, siitä voi Suomessa silti saada opintotukea</p><p>Palkaton harjoittelu voi kuitenkin lisätä eriarvoisuutta eri taustoista tulevien nuorten välillä. Kaikilla ei yksinkertaisesti ole varaa osallistua harjoitteluun ilman palkkaa. Lisäksi joillakin aloilla palkaton harjoittelu on niin yleistä, että se vaikeuttaa jo valmistuneidenkin työllistymistä</p><p>Toisaalta joissakin tapauksissa palkattomien harjoittelun kieltäminen saattaisi vähentää tarjolla olevien harjoittelupaikkojen määrää</p><p>EU on jo päättänyt määrätä työharjoitteluille yhteisiä vaatimuksia. Nämä koskevat harjoittelun kestoa, palkkausta ja sosiaaliturvaa. EU-parlamentti voi vaikuttaa näihin vaatimuksiin ja huolehtia siitä, että palkattom

In [ ]:
n_examples = 20
prompt_template = """
# Ohjeet

1. Lue alla olevat esimerkit vaalikoneen väittämistä, joista käyttä voi olla joko samaa tai eri mieltä, sekä niihin liittyvät lyhyet taustatiedot.
2. Luo kohdassa 'Uusi väittämä' olevalle väittämälle samanlainen taustatietoteksti.
3. Luo väittämälle lisäksi lyhyt, muutaman lauseen teksti, joka kertoo, mikä väittämässä esitetyn asian tila on tällä hetkellä.
4. Jos väittämässä esiintyy termejä, jotka saattavat olla tavallisille ihmisille epäselviä, lisää yhden lauseen selitys jokaiselle termille.
5. Esitä nämä teksti alla olevan formaatin mukaisesti.

# Tulosformaatti

## [Väittämä]

### Taustatiedot

[Taustietoteksti]

### Nykytila

[Asian nykytila]

### Termit
- [Termi 1]: [Termin 1 selitys]
- [Termi 2]: [Termin 2 selitys]
- jne.

# Esimerkit taustietoteksteistä
"""
for i in range(min(n_examples, len(example_data))):
  item = example_data.iloc[0]
  text = item['text']['fi']
  info = item['info']['fi']
  prompt_template += f"""

  ## Esimerkki {i+1}

- Väittämä: {text}

- Taustatiedot: {info}
"""

prompt_template += """

# Uusi väittämä

Luo alla olevalle väittämälle ohjeissa määritetyt tekstit ja näytä ne tulosformaatin mukaisesti.

- Väittämä: {statement}
"""

print(prompt_template)


# Ohjeet

1. Lue alla olevat esimerkit vaalikoneen väittämistä, joista käyttä voi olla joko samaa tai eri mieltä, sekä niihin liittyvät lyhyet taustatiedot.
2. Luo kohdassa 'Uusi väittämä' olevalle väittämälle samanlainen taustatietoteksti.
3. Luo väittämälle lisäksi lyhyt, muutaman lauseen teksti, joka kertoo, mikä väittämässä esitetyn asian tila on tällä hetkellä.
4. Jos väittämässä esiintyy termejä, jotka saattavat olla tavallisille ihmisille epäselviä, lisää yhden lauseen selitys jokaiselle termille.
5. Esitä nämä teksti alla olevan formaatin mukaisesti.

# Tulosformaatti

## [Väittämä]

### Taustatiedot

[Taustietoteksti]

### Nykytila

[Asian nykytila]

### Termit
- [Termi 1]: [Termin 1 selitys]
- [Termi 2]: [Termin 2 selitys]
- jne.

# Esimerkit taustietoteksteistä


## Esimerkki 1

- Väittämä: Opintoihin kuuluvasta työharjoitteluista pitää aina maksaa palkkaa EU:ssa.

- Taustatiedot: <p>Moniin opintoihin kuuluu työharjoittelua. Se voi Suomessa olla tällä hetkellä palkallista t

## Read input data

In [5]:
input_data = pd.read_csv('input/local-elections-2025-staments.tsv', sep='\t')
input_data.head()

,statement
0,Äänestysikärajaa tulee laskea 16 ikävuoteen ku...
1,Nuorisovaltuustoille tulee aina antaa puhe- ja...
2,Kunnan ja hyvinvointialueen palveluita tulee t...
3,"Julkisissa tarjoiluissa, kuten kouluruokailuss..."
4,Kunnassani tulisi tehdä enemmän toimia seksuaa...


In [6]:
input_data.statement.to_list()

['Äänestysikärajaa tulee laskea 16 ikävuoteen kunta- ja aluevaaleissa.',
 'Nuorisovaltuustoille tulee aina antaa puhe- ja läsnäolo-oikeus nuoria koskevassa päätöksenteossa esimerkiksi valtuustossa tai lautakunnissa.',
 'Kunnan ja hyvinvointialueen palveluita tulee tarjota kunnan ja hyvinvointialueen virallisten kielien lisäksi myös englanniksi.',
 'Julkisissa tarjoiluissa, kuten kouluruokailussa ja sairaaloissa, ei tule luopua lihan tarjoamisesta.',
 'Kunnassani tulisi tehdä enemmän toimia seksuaali- ja sukupuolivähemmistöihin kuuluvien nuorten syrjinnän ehkäisemiseksi.',
 'Kunnan tulee mahdollistaa jokaiselle alle 29-vuotiaalle mielekäs ja maksuton harrastus.',
 'Kouluväkivaltaan pitäisi puuttua mieluummin poliisin kuin sosiaalitoimen toimesta peruskouluissa.',
 'Nuoria ja nuoria aikuisia tulee priorisoida kunnan taloudellisessa päätöksenteossa.',
 'Kunnan on tuettava nuorten aikuisten työllistymistä ja sitä tukevia palveluita tarvittaessa muiden väestöryhmien edelle.',
 'Toisen astee